This notebook takes a look at the core ML pipeline.

In [4]:
import torch

In [87]:
# Let's create a densely-connected neural net to train.
# Each node in the network will belong a layer.
# Each input x_ij (layer i, node j) will be the weighted sum of activations from the previous layer, plus a bias term
# Its output a_ij (layer i, node j) will be f( x_ij ), where f is an activation function.

class Node:
    def __init__(self, weights, bias, activation_fn):
        self.weights = weights
        self.bias = bias
        self.activation_fn = activation_fn
        
        self.activation = 0 # activation for a given datapoint, calculated in forward pass
        self.delta = 0 # delta for back-prop, calculated in backward pass

def weighted_sum(weights, activations, bias):
    return torch.dot( weights, activations ) + bias;

def get_activations(model, layer_index):
    activations = [];
    for node in model[layer_index -1 ].nodes:
        activations.append( node.activation )
    return torch.tensor(activations)

class Layer:
    def __init__(self, num_weights_per_node, num_nodes, activation_fn):
        self.nodes = []
        for j in range(num_nodes):
            self.nodes.append( Node( torch.rand(num_weights_per_node), torch.rand(1), activation_fn ) )

model = [
    Layer( 4, 3, "relu"),
    Layer( 3, 3, "relu"),
    Layer( 3, 2, "relu")
]

def relu_derivative(a):
    return 1.0 if a > 0 else 0.0

# Run forward pass on a single data point, data_k
def forward_prop_k(model, data_k):
    # data_k will be a rank-1 tensor with the same shape as the first layer
    for i in range(len(model)):
        layer_i = model[i]
        activations = get_activations(model, i) if i > 0 else data_k
        for j in range(len(layer_i.nodes)):
            node_ij = layer_i.nodes[j]
            x_ij = weighted_sum(node_ij.weights, activations, node_ij.bias)
            a_ij = torch.nn.functional.relu(torch.tensor([x_ij])) # TODO: Use node_ij.activation_fn
            node_ij.activation = a_ij

# Run backwards propagation for a single data point, data_k
def back_prop_k(model, data_k, target_k):

    # Start with the output layer
    output_nodes = model[ len(model) - 1 ].nodes
    for o in range(len(output_nodes)):
        output_node = output_nodes[o]
        error = target_k[o] - output_node.activation
        df_dx = relu_derivative( output_node.activation ) # derivative of activation_fn, evaluated at stored activation
        output_node.delta = -2 * error * df_dx

    # Then, progress backwards through the layers
    
    return None
    

In [70]:
data_k = torch.rand(4)

In [76]:
forward_prop_k(model, data_k)

In [77]:
for node in model[1].nodes:
    print(node.activation)

tensor([1.6776])
tensor([2.5783])
tensor([2.6472])


In [88]:
back_prop_k(model, data_k)

2
1
0
